# Cleaning, preparing and selection of ASD cohort 

This markdown shows the process to obtain our sample cohort of patients with ASD. We first, clean and prepared our dataset to then, select the cohort following certain requirements. We mainly used the National Drug Codes (NDCs), the patient's ID, the ICD-9/ICD-10 list selected from Feroe et al. (2021) (<https://github.com/hms-dbmi/medicationUsageASDcomorbidities>), and evaluated for our specialized physicians. Finally, we also used the date of service, and the dates of enrollment.

## Load libraries

In [ ]:
library(data.table)
library(dplyr)
library(stringi)
library(stringr) 
library(tidyr) 
library(purrr)

## Index in the first diagnosis of ASD for each patient

We index in the first ASD diagnosis according with a list of ICD-9/ICD-10 codes.

In [ ]:
#ICD-9/ICD-10 related to ASD
ASD_diagnoses_code <- c("F84","F840","F843","F845","F849","F848","R419", 
                        "2990","29900","29901","2998","29980","29981","2999",
                        "29990","29991","299","2991","29910","29911")

#List of ASD diagnosis and their respective date 
ASD_diagnoses_codes_dates <- claims_ASD_wNDC_csv %>%
  select(pat_id, from_dt, starts_with("diag")) %>%
  select(-diagprc_ind) %>%
  pivot_longer(
    cols = -c(pat_id,from_dt), 
    names_to = "diagn_code",
    values_to = "code"
  ) %>% 
  select(-diagn_code) %>%
  #Filter the ICD related to ASD
  filter(code %in% ASD_diagnoses_code)  %>% 
  arrange(from_dt, pat_id) %>% 
  distinct()

#List of ASD diagnosis and respective year
ASD_diagnoses_codes_years <- ASD_diagnoses_codes_dates %>%
  mutate(year = substr(from_dt, 1, 4)) %>% 
  select(-from_dt) %>% 
  distinct() %>% 
  left_join(., select(demographicASD, pat_id, der_yob), by = "pat_id") %>% 
  mutate(relative_age = as.numeric(year) - der_yob)


#Obtain the first date for each diagnosis per patient  
first_date_ASD_Diagn <- ASD_diagnoses_codes_dates %>%
  group_by(pat_id) %>% 
  filter(row_number()==1) %>% 
  mutate(from_dt = as.Date(from_dt, "%Y-%d-%m")) %>% 
  ungroup()

After obtaining the date of the first ASD diagnosis, index the claims.

In [ ]:

'claims_ASD_wNDC_csv$from_dt <- as.Date(claims_ASD_wNDC_csv$from_dt, "%Y-%d-%m") # Join the dataframes and filter 
claims_afterDiagn_ASAD <- claims_ASD_wNDC_csv %>% left_join(first_date_ASD_Diagn, by = "pat_id", suffix = c("_claim", "_first_diagn")) %>% filter(from_dt_claim >= from_dt_first_diagn)

print(paste0("Claims from patients with claims since the first diagnosis (2006-2022): ", nrow(claims_afterDiagn_ASAD)) )

print(paste0("Patients with at least 1 ICD-9/ICD-10 codes for ASD (2006-2022): ",length(unique(claims_afterDiagn_ASAD$pat_id))) )'


## Verification of continuous enrollment

Patients can be included if have an insurance coverage for at least 12 months. First, we identified the count of months enrolled for each patient, and filtered the patients with at least 12 months.

In [ ]:
#How many times in a year a patient appear enrolled 
'table_enroll_year <- demographicASD_enrll2 %>%
  #filter(.$pat_id %in% ThreeASD_diagnoses_codes_filter) %>% 
  mutate(year = substr(month_id, 1, 4)) %>% 
  group_by(pat_id, year) %>% 
  count %>% 
  ungroup() %>%
  #filter(.$year < 2022) %>%
  mutate(active_months = n) %>% 
  select(-n)

#How many claims a patient have per year 
observe_claims_all_total <- claims_afterDiagn_ASAD %>% 
  mutate(year = substr(month_id, 1, 4)) %>% 
  group_by(pat_id, year) %>% 
  count %>% 
  ungroup() %>%
  arrange(pat_id, year) %>% 
  mutate(count_all_claims = n) %>% 
  select(-n)

#Count the months of enrollment per each patient
CountOfMonths_enroll <- table_enroll_year %>% 
  group_by(pat_id) %>% 
  summarize(total_months = sum(active_months)) %>% 
  ungroup()

print(paste0("Patients before verif. of contin. enroll: ", length(unique(CountOfMonths_enroll$pat_id))))

patients_full_enroll <- CountOfMonths_enroll %>% 
  filter(total_months >= 12)

print(paste0("Patients after verif. of contin. enroll: ", length(unique(patients_full_enroll$pat_id))))'

We observe that some patients have insurance records before the year of birth provided. We verified how many patients face the same problem. First, extract the first record in the insurance enrollment file for each patient.

In [ ]:

patientsFirstEnrollRecord <- table_enroll_year %>% 
  arrange(pat_id, year) %>% 
  group_by(pat_id) %>% 
  slice(1) %>% 
  ungroup() %>% 
  left_join(., select(demographicASD, der_yob, pat_id), by = "pat_id")

PatientsWithNonCoherentInsurance <- patientsFirstEnrollRecord %>% 
  filter(der_yob > year) %>% 
  select(pat_id) %>% 
  unique %>% 
  unlist 

claims_afterDiagn_ASAD <- claims_afterDiagn_ASAD %>% 
  filter(! pat_id %in% PatientsWithNonCoherentInsurance)

print(paste0("Number of patients after delete the incoherent ones: ", length(unique(claims_afterDiagn_ASAD$pat_id))))
      

There were 176 patients with the year of birth greater than the first insurance enrollment record. We deleted them from the dataset before doing the filter of continuous enrollment.

Finally, we included the patients with at least 12 months of enrollment.

In [ ]:
'claims_afterDiagn_ASAD <- claims_afterDiagn_ASAD %>% 
  filter(pat_id %in% patients_full_enroll$pat_id)


print(paste0("Claims from patients with continuos enrollment (2006-2022): ", nrow(claims_afterDiagn_ASAD)) )

print(paste0("Patients with  continuos enrollment  (2006-2022): ",length(unique(claims_afterDiagn_ASAD$pat_id))) )'


## ICD-9/ICD-10 verification for ASD diagnosis

We included the patients with at least 2 ICD-9/ICD-10 codes related to ASD in different dates, not matter if it is the same diagnosis code. Here, we also select our study period to be from 2012 to 2021.

In [ ]:
'ASD_diagnoses_codes_dates <- claims_afterDiagn_ASAD %>%
  select(pat_id, from_dt_claim, starts_with("diag")) %>%
  select(-diagprc_ind) %>%
  pivot_longer(
    cols = -c(pat_id,from_dt_claim), 
    names_to = "diagn_code",
    values_to = "code"
  ) %>% 
  select(-diagn_code) %>%
  #Filter the ICD related to ASD
  filter(code %in% ASD_diagnoses_code)  %>% 
  arrange(from_dt_claim, pat_id) %>% 
  distinct()

#Obtain the ASD diagnosis per year 
ASD_diagnoses_codes_years <- ASD_diagnoses_codes_dates %>%
  mutate(year = substr(from_dt_claim, 1, 4)) %>% 
  select(-from_dt_claim) %>% 
  distinct() %>% 
  left_join(., select(demographicASD, pat_id, der_yob), by = "pat_id") %>% 
  mutate(relative_age = as.numeric(year) - der_yob)

#More than 3 ASD diagnoses codes for each patient 
TwoASD_diagnoses_codes_filter <- ASD_diagnoses_codes_dates %>% 
  group_by(pat_id) %>%
  summarise(diagnoses_count = n_distinct(from_dt_claim,code)) %>%
  #At least 2 ICD codes for each patient
  filter(diagnoses_count >= 2) %>%
  select(pat_id) %>% 
  unique %>% 
  unlist

#Filter those patients in the claims dataset 
claims_afterDiagn_ASAD <- claims_afterDiagn_ASAD %>% 
  filter(pat_id %in% TwoASD_diagnoses_codes_filter) %>% 
  #Just leave the claims between 2012 to 2021
  mutate(year = substr(month_id, 1, 4)) %>% 
  filter(year >= 2012 & year <= 2021)  %>% 
  select(-year)

print(paste0("Claims from patients with at least 2 ICD-9/ICD-10 codes for ASD (2012-2021): ", nrow(claims_afterDiagn_ASAD)) )

print(paste0("Patients with at least 2 ICD-9/ICD-10 codes for ASD (2012-2021): ",length(unique(claims_afterDiagn_ASAD$pat_id))) )'

## Patients with ASD aged 2 to 26 yo

Our cohort are going to consist only in children aged 0 to 26 years old.

In [ ]:
#Add column of year of birth to include only patients 0 to 26 years old 
claims_afterDiagn_ASAD <- left_join(claims_afterDiagn_ASAD, demographicASD, by = "pat_id") %>% 
  mutate(year_claim = as.numeric(substr(month_id, 1, 4))) %>% 
  mutate(relative_age = year_claim - as.numeric(der_yob)) %>% 
  filter(relative_age >= 0 & relative_age <= 26) %>% 
  select(-year_claim)


'print(paste0("Claims from patients aged 0 to 26 years (2012 to 2021) ", nrow(claims_afterDiagn_ASAD)) )

print(paste0("Patients aged 0 to 26 years  (2012 to 2021): ",length(unique(claims_afterDiagn_ASAD$pat_id))) )''


## Pharmacotherapy cohort

First, we defined the classification of medications and their respective names.

In [ ]:

names_isomnia<-c("TRAZODONE","GABAPENTIN","AMITRIPTYLINE","CLOMIPRAMINE","DESIPRAMINE")
name_agressive_behv<-c("LORAZEPAM", "DIAZEPAM","CLONAZEPAM","ALPRAZOLAM","CLOZAPINE","HALOPERIDOL",
                       "CHLORPROMAZINE", "CANNABIDIOL")
names_depres_anx_OCD<-c("BUPROPION","FLUOXETINE","SERTRALINE","FLUVOXAMINE","ESCITALOPRAM",
                        "PAROXETINE","CITALOPRAM","BUSPIRONE","AMITRIPTYLINE","CLOMIPRAMINE",
                        "DESIPRAMINE")
names_irrit<-c("OLANZAPINE","ZIPRASIDONE","RISPERIDONE","QUETIAPINE","ARIPIPRAZOLE", "LURASIDONE")
name_antiseiz<-c("OXCARBAZEPINE","LAMOTRIGINE","DIVALPROEX","TOPIRAMATE")
name_adhd_stim<-c("METHYLPHENIDATE","DEXMETHYLPHENIDATE","SERDEXMETHYLPHENIDATE",
                  "LISDEXAMFETAMINE","AMPHETAMINE","AMPHETAMINE-DEXTROAMPHETAMINE","DEXTROAMPHETAMINE", "ATOMOXETINE","VILOXAZINE")
name_adhd_agress<-c("GUANFACINE","CLONIDINE")

all_names <- unique(c(names_isomnia, name_adhd_agress, name_adhd_stim, 
                      name_agressive_behv, name_antiseiz,names_depres_anx_OCD, 
                      names_irrit))


Then, extract the name of the drug using the generic names from the Rx file (already joined in the claims dataset)

In [ ]:

'pharmacotherapy_claims <- claims_afterDiagn_ASAD %>% 
  mutate(medication = map_chr(str_split(generic_name, " "), 1)) %>% 
  filter(medication %in% all_names) %>% 
  mutate(medication = if_else(medication == "AMPHETAMINE","AMPHETAMINE-DEXTROAMPHETAMINE",medication)) %>% 
  #filter generic names that we do not need 
  filter(! generic_name %in% GenericNamesToFilter$FilterGenericName) %>% 
  left_join(.,drug_class_names, by = "medication") %>% 
  distinct()

print(paste0("Claims from patients aged 0 to 26 years receiving pharmacotherapy (2012-2021): ", nrow(pharmacotherapy_claims)) )

print(paste0("Patients aged 0 to 26 years receiving pharmacotherapy(2012-2021): ",length(unique(pharmacotherapy_claims$pat_id))) )'



Final working dataset contains:

-   Patients' ID

-   Demographic information

-   Prescription information

-   Diagnosis and procedure information

-   Dates of prescription